In [ ]:
!pip install rasterio

In [ ]:
import ee
from google.colab import auth
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re
import geemap
import ipywidgets as widgets
import googleapiclient
from googleapiclient.discovery import build
from google.oauth2 import service_account
import geopandas as gpd
from osgeo import gdal, ogr, osr
import shutil
import rasterio
import joblib
import numpy as np
import pandas as pd
# Autentica e inizializza GEE
ee.Authenticate()
ee.Initialize(project='ee-andreagonnelli06')

In [ ]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Definisci il rettangolo di interesse
rectangle = Map.draw_last_feature
rectangle.getInfo()
geometria = rectangle.geometry().getInfo()

coordinates = geometria['coordinates'][0]
coordinates1 = coordinates[0]
coordinates2 = coordinates[2]
rectangles = ee.Geometry.Rectangle([coordinates1, coordinates2])

# Funzione per ritagliare un'immagine in base al rettangolo di interesse
def clippa(image):
    return image.clip(rectangles)
# Funzione per mascherare le nuvole in immagini Sentinel-2
def maskS2clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )
    return image.updateMask(mask).divide(10000)
# Funzione per calcolare gli indici NDVI, EVI e NDII
def calculate_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    ndii = image.normalizedDifference(['B8', 'B11']).rename('NDII')
    return image.addBands([ndvi, evi, ndii])

# Funzione per filtrare e unire le collezioni in base alle date
def filter_and_merge_collections(start_date, end_date):
    S1_PRS = ee.ImageCollection('COPERNICUS/S1_GRD')\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
        .filter(ee.Filter.eq('instrumentMode', 'IW'))\
        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
        .filterBounds(rectangles)\
        .map(clippa)

    S1_PRS_pc = S1_PRS.reduce(ee.Reducer.percentile([25,50,75]))
    S1_PRS_pc = ee.Image(10).pow(S1_PRS_pc.divide(10))

    S1_PRS_pc_Feats = S1_PRS_pc.select(['VH_p50','VV_p50']).clip(rectangles)
    S1_PRS_pc_Feats = S1_PRS_pc_Feats.reproject(crs= 'EPSG:4326',scale= 30)

    PRS_VV_iqr = S1_PRS_pc_Feats.addBands((S1_PRS_pc.select('VV_p75').subtract(S1_PRS_pc.select('VV_p25'))).rename('VV_iqr'))
    PRS_VH_iqr = S1_PRS_pc_Feats.addBands((S1_PRS_pc.select('VH_p75').subtract(S1_PRS_pc.select('VH_p25'))).rename('VH_iqr'))
    PRS_VV_iqr = PRS_VV_iqr.float()
    PRS_VH_iqr = PRS_VH_iqr.float()

    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

    composite = s2.filterDate(start_date, end_date)\
                  .filterBounds(rectangles)\
                  .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 60))\
                  .map(maskS2clouds)\
                  .map(calculate_indices)\
                  .map(clippa)

    S2_composite = composite.mean().reproject(crs= 'EPSG:4326', scale= 30).float()
    S2_composite = S2_composite.select('B4','B8','B11','B12','NDVI','EVI','NDII')

    SRTM = ee.Image("USGS/SRTMGL1_003")
    elevation = SRTM.clip(rectangles).reproject(crs='EPSG:4326', scale=30).float()
    slope = ee.Terrain.slope(SRTM).clip(rectangles).reproject(crs='EPSG:4326', scale=30).float()

    canopy_ht = ee.ImageCollection("projects/meta-forest-monitoring-okw37/assets/CanopyHeight")\
    .filterBounds(rectangles)\
    .map(clippa)\
    .select('cover_code')\
    .first()\
    .float().reproject(crs= 'EPSG:4326', scale= 30)

    l4b = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY')\
    .filterDate(start_date, end_date)\
    .filterBounds(rectangles)\
    .select('agbd')\
    .mean()

    aboveground = l4b.select('agbd')
    maschera = aboveground.gt(0).mask()

    dataset = aboveground.updateMask(maschera).clip(rectangles).float()
    dataset1 = dataset.reproject(crs= 'EPSG:4326', scale=30)

    combined_image = S2_composite.addBands(PRS_VV_iqr).addBands(PRS_VH_iqr).addBands(elevation).addBands(slope).addBands(canopy_ht).addBands(dataset1)
    bands = ['B4','B8','B11', 'B12','NDVI','EVI','NDII', 'VV_iqr', 'VH_iqr', 'elevation', 'slope','cover_code']
    combined_image = combined_image.select(bands)
    display(combined_image)

    return combined_image

# Funzione per creare le immagini medie mensili
def create_monthly_composites(start_year, end_year):
    monthly_composites = []

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')

            combined_image = filter_and_merge_collections(start_date, end_date)
            if combined_image.bandNames().size().getInfo() > 0:  # Verifica se l'immagine ha bande
                monthly_composites.append(combined_image.set('year', year).set('month', month))

    return ee.ImageCollection.fromImages(monthly_composites)

# Widget per selezionare l'intervallo di anni
start_year_widget = widgets.BoundedIntText(
    value=2020,
    min=2019,
    max=2023,
    step=1,
    description='Start Year:',
)

end_year_widget = widgets.BoundedIntText(
    value=2020,
    min=2019,
    max=2023,
    step=1,
    description='End Year:',
)

# Pulsante per generare le immagini medie mensili
generate_button = widgets.Button(
    description='Generate Monthly Composites',
    button_style='success',
)

display(start_year_widget, end_year_widget, generate_button)

# Funzione per visualizzare le immagini medie mensili
def on_generate_button_clicked(b):
    start_year = start_year_widget.value
    end_year = end_year_widget.value

    global monthly_composites
    monthly_composites = create_monthly_composites(start_year, end_year)

    # Crea una mappa
    Map = geemap.Map()

    # Centra la mappa sull'area di interesse
    Map.centerObject(rectangles, zoom=8)

    # Aggiungi i layer delle immagini medie mensili alla mappa
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            composite = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()
            if composite and composite.bandNames().size().getInfo() > 0:  # Verifica se l'immagine ha bande
                layer_name = f'{year}-{month:02d} Composite'
                Map.addLayer(composite, {}, layer_name)

    display(Map)

generate_button.on_click(on_generate_button_clicked)

# Widget per selezionare l'intervallo di anni e mesi per il download
download_start_year_widget = widgets.BoundedIntText(
    value=2020,
    min=2020,
    max=2023,
    step=1,
    description='Start Year:',
)

download_start_month_widget = widgets.BoundedIntText(
    value=4,
    min=4,
    max=12,
    step=1,
    description='Start Month:',
)

download_end_year_widget = widgets.BoundedIntText(
    value=2020,
    min=2019,
    max=2023,
    step=1,
    description='End Year:',
)

download_end_month_widget = widgets.BoundedIntText(
    value=12,
    min=4,
    max=12,
    step=1,
    description='End Month:',
)

# Pulsante per scaricare le immagini
download_button = widgets.Button(
    description='Download Composites',
    button_style='info',
)

display(download_start_year_widget, download_start_month_widget, download_end_year_widget, download_end_month_widget, download_button)

# Funzione per scaricare le immagini medie mensili
def on_download_button_clicked(b):
    start_year = download_start_year_widget.value
    start_month = download_start_month_widget.value
    end_year = download_end_year_widget.value
    end_month = download_end_month_widget.value
    out_dir = 'MERGED_COLLECTIONS'
    for year in range(start_year, end_year + 1):
        for month in range(start_month, end_month + 1):
            composite = monthly_composites.filter(ee.Filter.eq('year', year)).filter(ee.Filter.eq('month', month)).first()
            if composite and composite.bandNames().size().getInfo() > 0:  # Verifica se l'immagine ha bande
                file_name = f'{year}_{month:02d}'
                geemap.ee_export_image_to_drive(composite, description=file_name, folder=out_dir, fileNamePrefix=file_name, scale=30, region=rectangles)
                print(f'{file_name} scaricato con successo.')

download_button.on_click(on_download_button_clicked)


BoundedIntText(value=2020, description='Start Year:', max=2023, min=2019)

BoundedIntText(value=2020, description='End Year:', max=2023, min=2019)

Button(button_style='success', description='Generate Monthly Composites', style=ButtonStyle())

BoundedIntText(value=2020, description='Start Year:', max=2023, min=2020)

BoundedIntText(value=4, description='Start Month:', max=12, min=4)

BoundedIntText(value=2020, description='End Year:', max=2023, min=2019)

BoundedIntText(value=12, description='End Month:', max=12, min=4)

Button(button_style='info', description='Download Composites', style=ButtonStyle())

Map(center=[42.179624104993174, 12.787441999994439], controls=(WidgetControl(options=['position', 'transparent…

2022_04 scaricato con successo.
2022_05 scaricato con successo.
2022_06 scaricato con successo.
2022_07 scaricato con successo.
2022_08 scaricato con successo.
2022_09 scaricato con successo.
2022_10 scaricato con successo.
2022_11 scaricato con successo.
2022_12 scaricato con successo.


In [ ]:

model = joblib.load('/content/drive/MyDrive/RF_AGB/rf.pkl')

In [ ]:

image_folder = '/content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS'
image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith('.tif')]


for image_path in image_paths:
    with rasterio.open(image_path) as src:
        img = src.read()  # Leggere tutte le bande
        profile = src.profile


In [ ]:

def handle_nan(img):
    for i in range(img.shape[0]):  # iterare su ciascuna banda
        nan_mask = np.isnan(img[i])
        if np.any(nan_mask):
            mean_value = np.nanmean(img[i])
            img[i][nan_mask] = mean_value
    return img

for image_path in image_paths:
    with rasterio.open(image_path) as src:
        img = src.read()  # Leggere tutte le bande
        profile = src.profile

    # Gestire i valori NaN
    img = handle_nan(img)

    # Rimuovere la banda indesiderata (indice 12)
    #if img.shape[0] > 12:  # Assicurarsi che ci sia una 13a banda da rimuovere
        #img = np.delete(img, 12, axis=0)
        #img = np.delete(img, 2, axis=0)


    # Trasporre l'immagine
    bands, height, width = img.shape
    img_reshaped = img.reshape(bands, -1).T
    img_reshaped

    # Fare previsioni
    predictions = model.predict(img_reshaped)

    # Rimodellare le previsioni
    predicted_image = predictions.reshape(height, width)

    # Aggiornare il profilo
    profile.update(dtype=rasterio.float32, count=1)

    # Salvare l'immagine predetta
    predicted_image_path = image_path.replace('.tiff', '_predicted.tiff')
    with rasterio.open(predicted_image_path, 'w', **profile) as dst:
        dst.write(predicted_image.astype(rasterio.float32), 1)
    with rasterio.open(predicted_image_path) as src:

    # Leggere tutte le bande
       image = src.read([1])
       transform = src.transform
       crs = src.crs

    print(f'Saved predicted image to {predicted_image_path}')


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s


Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_04.tif
Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_10.tif


[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s


Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_09.tif
Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_08.tif


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s


Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_07.tif
Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_06.tif


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s


Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_05.tif
Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_11.tif


[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 364 tasks      | elapsed:    0.1s


Saved predicted image to /content/drive/MyDrive/IMMAGINI_CO2/MERGED_COLLECTIONS/2022_12.tif


In [ ]:

# Percorso alla cartella contenente il file delle credenziali
credentials_folder = '/content/drive/MyDrive/credentials_folder'
credentials_file = os.path.join(credentials_folder, 'credentials.json')

# Funzione per autenticarsi a Google Drive
def authenticate_drive():
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = service_account.Credentials.from_service_account_file(credentials_file, scopes=SCOPES)
    return build('drive', 'v3', credentials=creds)

# Autenticazione a Google Drive
drive_service = authenticate_drive()

# Funzione per creare una directory se non esiste
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Funzione per scaricare un file da Google Drive
def download_file_from_drive(file_id, file_name, directory):
    request = drive_service.files().get_media(fileId=file_id)
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'wb') as f:
        downloader = googleapiclient.http.MediaIoBaseDownload(f, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")
    return file_path

# Widget per gli URL dei file
file_urls_widget = widgets.Textarea(
    value='',
    placeholder='Inserisci gli URL dei file di Google Drive, uno per riga',
    description='File URLs:',
    layout=widgets.Layout(width='100%', height='200px')
)

# Pulsante per scaricare i file
download_files_button = widgets.Button(
    description='Download Files from URLs',
    button_style='info',
)

display(file_urls_widget, download_files_button)

# Funzione per estrarre l'ID del file da un URL di Google Drive
def extract_file_id(url):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    return match.group(1) if match else None

# Funzione per convertire un raster a interi
def float2int(raster_path, output_path):
    src_ds = gdal.Open(raster_path)
    if src_ds is None:
        print(f"Unable to open {raster_path}")
        return None

    band = src_ds.GetRasterBand(1)
    arr = band.ReadAsArray()

    # Convert array to integers
    int_arr = arr.astype(int)

    # Create a new raster dataset
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_path, src_ds.RasterXSize, src_ds.RasterYSize, 1, gdal.GDT_Int32)
    out_ds.SetGeoTransform(src_ds.GetGeoTransform())
    out_ds.SetProjection(src_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(int_arr)

    out_band.FlushCache()
    return output_path

# Funzione per convertire un raster in shapefile
def raster2polygon(raster_path, shapefile_path):
    # Convertire il raster a interi
    int_raster_path = raster_path.replace('.tif', '_int.tif')
    int_raster_path = float2int(raster_path, int_raster_path)

    if int_raster_path is None:
        return

    src_ds = gdal.Open(int_raster_path)
    src_band = src_ds.GetRasterBand(1)

    # Create a new shapefile
    drv = ogr.GetDriverByName("ESRI Shapefile")
    if os.path.exists(shapefile_path):
        drv.DeleteDataSource(shapefile_path)
    out_ds = drv.CreateDataSource(shapefile_path)
    out_layer = out_ds.CreateLayer("polygonized", srs=osr.SpatialReference(wkt=src_ds.GetProjection()), geom_type=ogr.wkbPolygon)
    new_field = ogr.FieldDefn('values', ogr.OFTInteger)
    out_layer.CreateField(new_field)

    gdal.Polygonize(src_band, None, out_layer, 0, [], callback=None)

    out_ds = None
    src_ds = None
    print(f"Shapefile creato: {shapefile_path}")

# Funzione per processare le immagini scaricate e creare shapefiles con i calcoli richiesti
def process_images(image_paths, shapefiles_directory):
    create_directory(shapefiles_directory)

    for image_path in image_paths:
        file_name = os.path.basename(image_path)
        base_name = os.path.splitext(file_name)[0]
        shapefile_path = os.path.join(shapefiles_directory, f"{base_name}.shp")

        # Convertire l'immagine in shapefile e applicare i calcoli
        raster2polygon(image_path, shapefile_path)
        shapefile = gpd.read_file(shapefile_path)

        # Applicare i calcoli e aggiungere nuove colonne
        shapefile["BGB"] = shapefile["values"] * 0.2
        shapefile["BIOMASS"] = shapefile["values"] + shapefile["BGB"]
        shapefile["CARBON"] = shapefile["BIOMASS"] * 0.475
        shapefile["CO2"] = shapefile["CARBON"] * 3.67

        # Salvare shapefile modificato
        shapefile.to_file(shapefile_path)
        print(f"Shapefile con calcoli aggiornati: {shapefile_path}")

# Funzione per scaricare i file dagli URL inseriti
def on_download_files_button_clicked(b):
    file_urls = file_urls_widget.value.strip().split('\n')
    image_directory = '/content/drive/MyDrive/downloaded_images'
    shapefiles_directory = '/content/drive/MyDrive/shapefiles'

    create_directory(image_directory)
    create_directory(shapefiles_directory)

    image_paths = []

    for url in file_urls:
        file_id = extract_file_id(url)
        if file_id:

            file_name = f'{file_id}.tif'
            file_path = download_file_from_drive(file_id, file_name, image_directory)
            image_paths.append(file_path)
            print(f"File {file_name} scaricato con successo.")
        else:
            print(f"ID del file non trovato per URL: {url}")

    # Processare le immagini scaricate
    process_images(image_paths, shapefiles_directory)

    # Zip the shapefiles directory for download
    shutil.make_archive(shapefiles_directory, 'zip', shapefiles_directory)
    print("Shapefiles zipped for download.")

# Associa la funzione di scaricamento al pulsante
download_files_button.on_click(on_download_files_button_clicked)


Textarea(value='', description='File URLs:', layout=Layout(height='200px', width='100%'), placeholder='Inseris…

Button(button_style='info', description='Download Files from URLs', style=ButtonStyle())

Download 100%.
File 19AlTdvIB9ONJ8ymzOqLxrg8-lM2ni3wQ.tif scaricato con successo.
Download 100%.
File 1K-IlxnToBkX_qjs-F2jwFQQWPj-tS1ez.tif scaricato con successo.
Download 100%.
File 1wRDARUTyI_ez1UiLROWOkfvWlJnmTqRb.tif scaricato con successo.
Download 100%.
File 1TGVzp1Q6_Doq3MEZDUeJmsAcHBqb4wRq.tif scaricato con successo.
Download 100%.
File 1wMsYQoCas5aCLRc1D-TFiWzRCc5m8UZZ.tif scaricato con successo.
Download 100%.
File 1XLj9gUMUx20o57AbPk02DOnlGfF2x1Nj.tif scaricato con successo.
Download 100%.
File 1K6w5UW2Td5tAXRQlVySMvgSv0EiW45ch.tif scaricato con successo.
Download 100%.
File 1PLhZRyxtyAs3cqdzHn0WXUQqPolDWJwV.tif scaricato con successo.
Download 100%.
File 1ywlYdJma97yOdWcGXL39uUyJMaJZ4Ksh.tif scaricato con successo.
Shapefile creato: /content/drive/MyDrive/shapefiles/19AlTdvIB9ONJ8ymzOqLxrg8-lM2ni3wQ.shp
Shapefile con calcoli aggiornati: /content/drive/MyDrive/shapefiles/19AlTdvIB9ONJ8ymzOqLxrg8-lM2ni3wQ.shp
Shapefile creato: /content/drive/MyDrive/shapefiles/1K-IlxnToBkX_q

In [ ]:
import psutil
import os
import time

# Funzione per monitorare l'uso della memoria
def memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss  # In bytes

# Esempio di utilizzo durante l'inferenza
print(f"Memoria utilizzata: {memory_usage() / (1024 * 1024):.2f} MB")